In [ ]:
import cv2
from tqdm import tqdm
import os
from pathlib import Path

cap = cv2.VideoCapture("media.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

saved_frame_count = 0

frames_output_folder = "trackedmedia/rawintermediate"
Path(frames_output_folder).mkdir(exist_ok=True, parents=True)
for frame_id in tqdm(range(total_frames), desc="Unpacking video"):
    ret, frame = cap.read()
    if not ret:
        break

    if frame_id % 1 == 0:
        frame_path = os.path.join(
            frames_output_folder, f"frame{saved_frame_count:04d}.png")

        is_success, im_buf_arr = cv2.imencode(".jpg", frame)
        im_buf_arr.tofile(frame_path)

        saved_frame_count += 1
cap.release()
print(f"Extracted {saved_frame_count} frames")

In [ ]:
from transformers import AutoImageProcessor, DetrForObjectDetection

image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

In [ ]:
import os
import torch
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes, save_image
from PIL import Image

frames = os.listdir(frames_output_folder)
total_frames = len(frames)

tracked_frames_folder = 'trackedmedia/intermediate/'
Path(tracked_frames_folder).mkdir(exist_ok=True, parents=True)
for i in tqdm(range(total_frames), desc="Scanning frames for cars"):
    frame = f'{frames_output_folder}/frame{f'{i + 10000}'[1:]}.png'

    image = Image.open(frame).convert("RGB")

    inputs = image_processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    target_sizes = torch.tensor([image.size[::-1]])
    results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[
        0
    ]

    image = read_image(frame)
    image = (255.0 * (image - image.min()) / (image.max() - image.min())).to(torch.uint8)
    image = image[:3, ...]
    pred_boxes = results["boxes"].long()
    labels = [f"{model.config.id2label[label.item()]}: {score} " for label, score in zip(results["labels"], results["scores"])]
    output_image = draw_bounding_boxes(image, pred_boxes, labels, colors="red")
    save_image(output_image / 255, f"{tracked_frames_folder}/frame{f'{10000 + i}'[1:]}.png")


In [ ]:
import numpy as np
import subprocess

intermediate_file = "intermediate.mp4"
dims = Image.open(f'{frames_output_folder}/frame0000.png').size
if os.path.exists(intermediate_file):
    os.remove(intermediate_file)
vidwriter = cv2.VideoWriter(intermediate_file, cv2.VideoWriter_fourcc(*"mp4v"), fps, dims)

total_frames = len(os.listdir(tracked_frames_folder))
for i in tqdm(range(total_frames), desc="Assembly frames into video"):
    frame = Image.open(f'{tracked_frames_folder}/frame{f'{10000 + i}'[1:]}.png')
    vidwriter.write(np.asarray(frame))
vidwriter.release()

if os.path.exists("trackedmedia.mp4"):
    os.remove("trackedmedia.mp4")
# the media is corrupt in curtain file editors for whatever reason, running it through ffmpeg fixes this
subprocess.run(["ffmpeg", "-i", intermediate_file, "trackedmedia.mp4"])
os.remove(intermediate_file)